In [7]:
import subprocess
import json

from transformers.commands.serving import ToolState

# Start the subprocess
proc = subprocess.Popen(
    ['uv', 'run', 'tools_server.py'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    text=True
)

# Prepare the command
command = {
    "method": "tools/call",
    "params": {
        "name": "get_current_price",
        "arguments": {"pair": "BTC/USD"}
    }
}

command_json = json.dumps(command)
print(f"Sending command: {command_json}")

# Send the command
proc.stdin.write(command_json + '\n')
proc.stdin.flush()


for line in proc.stdout:
    try:
        result = json.loads(line)
        print(f"Received reply: {result}")
        break  # Stop after first valid JSON
    except json.JSONDecodeError:
        print(line) # Print non-JSON lines
        continue

proc.terminate()

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Sending command: {"method": "tools/call", "params": {"name": "get_current_price", "arguments": {"pair": "BTC/USD"}}}
Fetching current price for BTC/USD...

Received reply: {'result': '{"pair": "BTC/USD", "base": "BTC", "quote": "USD", "price": 58869.32, "timestamp": "2025-09-28T07:41:56.462009"}'}


In [8]:

from basic.part_06_universal_function_call.tools_server_client import ToolsServerClient

with ToolsServerClient('tools_server.py') as tools_client:
    result = tools_client.call_function('get_current_price', {'pair': 'BTC/USD'})
    print(result)

{'result': '{"pair": "BTC/USD", "base": "BTC", "quote": "USD", "price": 10588.16, "timestamp": "2025-09-28T07:42:01.211545"}'}


## How do we know the tools?

In [9]:
from basic.part_06_universal_function_call.tools_server_client_improved import ToolsServerClientImproved

with ToolsServerClientImproved('tools_server_improved.py') as tools_client:
    result = tools_client.list_tools()
    print(result)

[{'type': 'function', 'function': {'name': 'get_current_price', 'description': 'Get the current price of a cryptocurrency trading pair', 'parameters': {'type': 'object', 'properties': {'pair': {'type': 'string', 'description': 'The trading pair in format BASE/QUOTE, e.g. BTC/USD, ETH/USD'}}, 'required': ['pair'], 'additionalProperties': False}}}]


In [10]:
from basic.api_key import ensure_api_key
ensure_api_key()
from openai import OpenAI
client = OpenAI()


with ToolsServerClientImproved('tools_server_improved.py') as tools_client:
    tools = tools_client.list_tools()

    messages = [
        {"role": "system", "content": "You are a helpful assistant that is good with finance."},
        {"role": "user", "content": "What is the current price of Bitcoin in USD?"},
    ]
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        tools=tools
    )

    response_message = response.choices[0].message
    messages.append(response_message)

    if response_message.tool_calls:
        tool_call = response_message.tool_calls[0]
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        function_call_result = tools_client.call_function(function_name, function_args)
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": function_call_result["result"]
        })
        resp = client.chat.completions.create(
            model="gpt-4.1-mini",
            messages=messages
        )
        resp_message = resp.choices[0].message
        print(resp_message.content)
        messages.append({"role": "assistant", "content": resp_message.content})


OpenAI API key has been configured.
The current price of Bitcoin (BTC) in USD is approximately $58,705.66.
